In [1]:
import sys
sys.path.append('../')
from poker_env_multi import PokerEnvMulti
from agents.random_policy import RandomActions
from agents.heuristic_policy import HeuristicPolicy
from ray.rllib.algorithms.ppo import PPOConfig
from gym import spaces
import mpu
import numpy as np
import ray

from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env

/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:28: DeprecationWarning: HAMMING is dep

In [2]:
def select_policy(agent_id, episode, **kwargs):
    if agent_id == 0:
        return "learned"
    elif agent_id == 1:
        return "Heuristic_10"
    elif agent_id == 2:
        return "Heuristic_100"
    elif agent_id == 3:
        return "Heuristic_1000"
    return "Heuristic_1000"

def env_creator(config):
    env = PokerEnvMulti(select_policy, config)
    return env

register_env("poker", lambda config: env_creator(config))

/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
heuristic_observation_space = spaces.Dict({
            "hand": spaces.Box(0, 1, shape=(28, )),
            "community": spaces.Box(0, 1, shape=(28, ))
        })
action_space = spaces.Discrete(3)

config = (
    PPOConfig()
    .rollouts(
        num_rollout_workers=0,
        num_envs_per_worker=1,
    )
    .training(
        train_batch_size=128,
        gamma=0.99,
    )
    .multi_agent(
        policies={
            "random": PolicySpec(policy_class=RandomActions),
            "Heuristic_10": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 0}),
            "Heuristic_100": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 1}),
            "Heuristic_1000": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 2}),
            "learned": PolicySpec(
                config={}
            ),
        },
        policy_mapping_fn=select_policy,
        policies_to_train=['learned'],
    )
    .resources(num_gpus=0)\
)
trainer = config.build(env="poker")


2022-11-07 17:39:02,342	WARNING ppo.py:350 -- `train_batch_size` (128) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 128.
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ray/_private/ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.1/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
2022-11-07 17:39:05,799	INFO worker.py:1518 -- Started a local Ray instance.
2022-11-07 17:39:08,151	WARNING env.py:235 -- Your MultiAgentEnv <PokerEnvMulti instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__` from within your MutiAgentEnv's const

In [9]:

trainer.train()

progress
showdown
done
{'hands': ['KING of DIAMONDS; TWO of SPADES; ', 'KING of CLUBS; THREE of DIAMONDS; ', 'KING of HEARTS; THREE of CLUBS; ', 'TWO of HEARTS; THREE of HEARTS; '], 'community': 'ACE of CLUBS; TWO of CLUBS; QUEEN of SPADES; TWO of DIAMONDS; JACK of HEARTS; ', 'scores': array([[3., 1., 6.],
       [0., 0., 0.],
       [0., 0., 0.],
       [3., 1., 5.]]), 'winners': 0, 'chips': [65, 138, 97, 98], 'bets': array([[2., 1., 1., 1.],
       [2., 1., 0., 0.],
       [0., 0., 0., 0.],
       [2., 1., 1., 1.]])}
[  0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.  65. 137.  95.  96.]
progress
progress
progress
progress
showdown
done
{'hands': ['JACK of DIAMONDS; TWO of C

{'custom_metrics': {},
 'episode_media': {},
 'num_recreated_workers': 0,
 'info': {'learner': {'learned': {'learner_stats': {'cur_kl_coeff': 0.10000000149011612,
     'cur_lr': 4.999999873689376e-05,
     'total_loss': 1.7376896,
     'policy_loss': 0.04304852,
     'vf_loss': 1.6927345,
     'vf_explained_var': -0.0011304895,
     'kl': 0.01906628,
     'entropy': 1.054242,
     'entropy_coeff': 0.0,
     'model': {}}}},
  'num_env_steps_sampled': 640,
  'num_env_steps_trained': 640,
  'num_agent_steps_sampled': 637,
  'num_agent_steps_trained': 637},
 'sampler_results': {'episode_reward_max': 11.0,
  'episode_reward_min': -5.0,
  'episode_reward_mean': 2.1025641025641026,
  'episode_len_mean': 16.256410256410255,
  'episode_media': {},
  'episodes_this_iter': 8,
  'policy_reward_min': {'learned': -8.0,
   'Heuristic_10': -8.0,
   'Heuristic_100': -9.0,
   'Heuristic_1000': -10.0},
  'policy_reward_max': {'learned': 13.0,
   'Heuristic_10': 18.0,
   'Heuristic_100': 16.0,
   'Heurist

In [5]:
a = []
a = np.zeros(16)
a = np.concatenate([a, np.zeros(8)])
a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])